In [163]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from env import get_connection

pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings('ignore')

In [3]:
url = get_connection('curriculum_logs')

In [116]:
df = pd.read_sql('''
            SELECT * FROM logs as l
            JOIN cohorts as c
            WHERE l.cohort_id = c.id;
            ''', url)

In [117]:
df

,date,time,path,user_id,cohort_id,ip,id,name,slack,start_date,end_date,created_at,updated_at,deleted_at,program_id
0,2018-01-26,09:55:03,/,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,#hampton,2015-09-22,2016-02-06,2016-06-14 19:52:26,2016-06-14 19:52:26,None,1
4,2018-01-26,09:56:24,javascript-i/conditionals,2,22.0,97.105.19.61,22,Teddy,#teddy,2018-01-08,2018-05-17,2018-01-08 13:59:10,2018-01-08 13:59:10,None,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847325,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2
847326,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2
847327,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2
847328,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,28,Staff,#,2014-02-04,2014-02-04,2018-12-06 17:04:19,2018-12-06 17:04:19,None,2


In [118]:
df = df[df['program_id']!=4]

In [119]:
df.drop(columns=['slack', 'deleted_at','created_at','updated_at'], inplace=True)

/var/folders/mp/b9xckkh51ds5b9r44vtn7w7w0000gn/T/ipykernel_61647/3884105429.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=['slack', 'deleted_at','created_at','updated_at'], inplace=True)


In [120]:
df = df[df['path']!='/']

In [121]:
df[df['date']>df['end_date']]

,date,time,path,user_id,cohort_id,ip,id,name,start_date,end_date,program_id
1,2018-01-26,09:56:02,java-ii,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1
2,2018-01-26,09:56:05,java-ii/object-oriented-programming,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1
3,2018-01-26,09:56:06,slides/object_oriented_programming,1,8.0,97.105.19.61,8,Hampton,2015-09-22,2016-02-06,1
31,2018-01-26,10:14:51,javascript-i,11,1.0,97.105.19.61,1,Arches,2014-02-04,2014-04-22,1
33,2018-01-26,10:15:02,javascript-i/functions,11,1.0,97.105.19.61,1,Arches,2014-02-04,2014-04-22,1
...,...,...,...,...,...,...,...,...,...,...,...
847325,2021-04-21,16:41:51,jquery/personal-site,64,28.0,71.150.217.33,28,Staff,2014-02-04,2014-02-04,2
847326,2021-04-21,16:42:02,jquery/mapbox-api,64,28.0,71.150.217.33,28,Staff,2014-02-04,2014-02-04,2
847327,2021-04-21,16:42:09,jquery/ajax/weather-map,64,28.0,71.150.217.33,28,Staff,2014-02-04,2014-02-04,2
847328,2021-04-21,16:44:37,anomaly-detection/discrete-probabilistic-methods,744,28.0,24.160.137.86,28,Staff,2014-02-04,2014-02-04,2


In [122]:
df['count'] = df.groupby('path')['path'].transform('count')

In [185]:
pd.DataFrame(df[df['count']<5]['ip'].value_counts())

,ip
97.105.19.58,540
97.105.19.61,46
136.50.20.17,41
192.171.117.210,33
173.174.142.84,28
...,...
70.115.238.206,1
107.77.217.152,1
67.11.136.146,1
67.10.152.210,1


In [147]:
cohort_path_1 = pd.DataFrame(df['path'].drop_duplicates())
cohort_path_2 = pd.DataFrame(df['path'].drop_duplicates())
cohort_path_3 = pd.DataFrame(df['path'].drop_duplicates())

In [148]:
prgm_1 = df[df['program_id']==1]['name'].unique()
prgm_2 = df[df['program_id']==2]['name'].unique()
prgm_3 = df[df['program_id']==3]['name'].unique()

In [149]:
for name in prgm_1:
    cohort_path_1 = pd.merge(cohort_path_1, pd.DataFrame({f'{name}':df[df['name']==f'{name}'].value_counts('path')}), 
         how='left',on='path')
    
    cohort_path_1[f'{name}'] = cohort_path_1[f'{name}'].fillna(0)

for name in prgm_2:
    cohort_path_2 = pd.merge(cohort_path_2, pd.DataFrame({f'{name}':df[df['name']==f'{name}'].value_counts('path')}), 
         how='left',on='path')
    
    cohort_path_2[f'{name}'] = cohort_path_2[f'{name}'].fillna(0)
    
for name in prgm_3:
    cohort_path_3 = pd.merge(cohort_path_3, pd.DataFrame({f'{name}':df[df['name']==f'{name}'].value_counts('path')}), 
         how='left',on='path')
    
    cohort_path_3[f'{name}'] = cohort_path_3[f'{name}'].fillna(0)

In [164]:
cohort_path_1['diff'] = cohort_path_1.max(axis=1) - cohort_path_1.min(axis=1)
cohort_path_2['diff'] = cohort_path_2.max(axis=1) - cohort_path_2.min(axis=1)
cohort_path_3['diff'] = cohort_path_3.max(axis=1) - cohort_path_3.min(axis=1)

In [183]:
cohort_path_1[cohort_path_1['diff']>200]

,path,Hampton,Arches,Quincy,Kings,Lassen,Glacier,Denali,Joshua,Olympic,Badlands,Ike,Franklin,diff
8,javascript-i,36.0,294.0,6.0,17.0,233.0,11.0,0.0,1.0,128.0,1.0,7.0,2.0,294.0
15,spring,43.0,192.0,8.0,3.0,222.0,2.0,0.0,0.0,24.0,0.0,5.0,2.0,222.0
20,index.html,1.0,1.0,6.0,84.0,877.0,0.0,0.0,8.0,28.0,0.0,6.0,0.0,877.0
34,html-css,21.0,215.0,12.0,10.0,174.0,21.0,0.0,5.0,69.0,2.0,12.0,1.0,215.0
84,javascript-ii,31.0,204.0,7.0,10.0,107.0,6.0,0.0,0.0,58.0,0.0,4.0,2.0,204.0
131,java-iii,57.0,153.0,3.0,8.0,224.0,0.0,0.0,1.0,25.0,0.0,5.0,3.0,224.0


In [179]:
cohort_path_2[cohort_path_2['diff']>1200]

,path,Teddy,Sequoia,Niagara,Pinnacles,Mammoth,Ulysses,Voyageurs,Wrangell,Xanadu,Yosemite,Staff,Zion,Andromeda,Betelgeuse,Ceres,Deimos,Europa,Fortuna,Apex,Ganymede,Hyperion,Bash,Jupiter,Kalypso,Luna,Marco,Neptune,Oberon,diff
0,java-ii,509.0,129.0,17.0,50.0,10.0,425.0,756.0,325.0,502.0,274.0,1260.0,624.0,398.0,481.0,681.0,511.0,375.0,657.0,666.0,641.0,688.0,268.0,755.0,417.0,255.0,40.0,5.0,0.0,1260.0
8,javascript-i,615.0,202.0,23.0,69.0,6.0,526.0,884.0,560.0,722.0,452.0,1817.0,897.0,516.0,717.0,1003.0,896.0,506.0,785.0,669.0,760.0,910.0,374.0,926.0,631.0,676.0,907.0,272.0,136.0,1811.0
15,spring,744.0,364.0,65.0,56.0,12.0,409.0,650.0,293.0,454.0,381.0,1403.0,672.0,487.0,299.0,562.0,533.0,324.0,636.0,600.0,508.0,526.0,311.0,621.0,339.0,74.0,43.0,8.0,2.0,1401.0
16,appendix,505.0,170.0,20.0,25.0,9.0,176.0,394.0,209.0,325.0,144.0,1288.0,296.0,161.0,188.0,411.0,343.0,243.0,356.0,450.0,561.0,400.0,174.0,421.0,180.0,92.0,82.0,15.0,10.0,1279.0
34,html-css,241.0,168.0,6.0,56.0,3.0,562.0,528.0,411.0,597.0,384.0,1284.0,675.0,375.0,457.0,766.0,773.0,353.0,585.0,738.0,504.0,439.0,265.0,784.0,495.0,307.0,479.0,313.0,21.0,1281.0
131,java-iii,732.0,105.0,29.0,45.0,13.0,412.0,770.0,337.0,497.0,351.0,1393.0,753.0,433.0,472.0,682.0,523.0,451.0,786.0,663.0,651.0,701.0,324.0,795.0,456.0,259.0,40.0,9.0,1.0,1392.0
610,toc,83.0,36.0,8.0,12.0,1.0,89.0,272.0,991.0,443.0,697.0,911.0,1465.0,638.0,610.0,911.0,750.0,964.0,1293.0,810.0,822.0,1022.0,404.0,1866.0,495.0,373.0,910.0,451.0,101.0,1865.0
617,search/search_index.json,103.0,45.0,7.0,9.0,1.0,142.0,328.0,504.0,577.0,361.0,1349.0,700.0,318.0,761.0,1380.0,746.0,742.0,1020.0,1497.0,1095.0,670.0,660.0,998.0,467.0,405.0,241.0,77.0,9.0,1496.0


In [181]:
cohort_path_3[cohort_path_3['diff']>600]

,path,Bayes,Curie,Darden,Easley,Florence,diff
1391,1-fundamentals/modern-data-scientist.jpg,650.0,467.0,470.0,7.0,61.0,643.0
1406,1-fundamentals/1.1-intro-to-data-science,640.0,461.0,460.0,8.0,64.0,632.0
1407,1-fundamentals/AI-ML-DL-timeline.jpg,648.0,465.0,470.0,7.0,61.0,641.0
1766,fundamentals/modern-data-scientist.jpg,24.0,100.0,308.0,379.0,627.0,603.0
1781,sql/mysql-overview,27.0,125.0,774.0,295.0,203.0,747.0
1795,classification/overview,35.0,136.0,1109.0,445.0,60.0,1074.0
2003,classification/scale_features_or_not.svg,28.0,96.0,943.0,463.0,60.0,915.0
